In [1]:
import os
import os.path as op
import json
import cv2
import base64
import numpy as np
from tqdm import tqdm

import numpy as np
import matplotlib.pyplot as plt

from PIL import Image, ImageDraw

from maskrcnn_benchmark.structures.tsv_file_ops import tsv_reader, tsv_writer
from maskrcnn_benchmark.structures.tsv_file_ops import generate_linelist_file
from maskrcnn_benchmark.structures.tsv_file_ops import generate_hw_file
from maskrcnn_benchmark.structures.tsv_file import TSVFile
from maskrcnn_benchmark.data.datasets.utils.image_ops import img_from_base64

In [14]:
orig_root = '../../../../datasets/coco_2014'
img_root = op.join(orig_root, 'images')

exp_root = '../../../../datasets/coco_2014_nn'

splits = ['train', 'val', 'test']

In [7]:
with open(op.join(orig_root, 'dataset_coco.json')) as f:
    karpathy = json.load(f)

In [25]:
print(karpathy['images'][555]['filepath'])
print(karpathy['images'][555]['filename'])
print(karpathy['images'][555]['cocoid'])

val2014
COCO_val2014_000000382088.jpg
382088


In [16]:
img_list = {split: [] for split in splits}

for sample in karpathy['images']:
    split = sample['split']
    if split == 'restval':
        continue
    filepath = sample['filepath']
    filename = sample['filename']
    img_key = str(sample['cocoid'])
    img_list[split].append(
        (img_key, op.join(filepath, filename))
    )
    
print(img_list['train'][0])  # sample
print([len(img_list[split]) for split in splits])  # train, val, test

('57870', 'train2014/COCO_train2014_000000057870.jpg')
[82783, 5000, 5000]


In [23]:
for split in splits:
    rows = []
    rows_label = []
    rows_hw = []

    # i = 5
    for sample in tqdm(img_list[split]):
        # print(sample)
        img_key = sample[0]
        img_path = op.join(img_root, sample[1])
        img = cv2.imread(img_path)
        img_encoded_str = base64.b64encode(cv2.imencode('.jpg', img)[1])
        row = (img_key, img_encoded_str)
        # print(row[1][800:900] , flush=True)
        rows.append(row)

        height = img.shape[0]
        width = img.shape[1]
        row_hw = [img_key, json.dumps([{'height': height, 'width': width}])]
        rows_hw.append(row_hw)
        
    exp_encoded_img_file = op.join(exp_root, f'{split}.img.tsv')
    exp_hw_file = op.join(exp_root, f'{split}.hw.tsv')
    tsv_writer(rows, exp_encoded_img_file)
    tsv_writer(rows_hw, exp_hw_file)
    # tsv_writer(rows_label, label_file)

100%|██████████| 5000/5000 [02:19<00:00, 35.80it/s]
